# Import Libraries

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# Loading the Data and Selecting the Features

### Import Data

In [2]:
loan_data_inputs_train = pd.read_csv('loan_data_inputs_train.csv',index_col=0)
loan_data_inputs_test = pd.read_csv('loan_data_inputs_test.csv',index_col=0)
loan_data_targets_train = pd.read_csv('loan_data_targets_train.csv',index_col=0)
loan_data_targets_test = pd.read_csv('loan_data_targets_test.csv',index_col=0)

### Explore Data 

In [3]:
loan_data_inputs_train.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,mths_since_last_delinq_factor : 52-79,mths_since_last_delinq_factor : > 79,mths_since_last_record_factor : Missing,mths_since_last_record_factor : <4,mths_since_last_record_factor : 4-33,mths_since_last_record_factor : 33-37,mths_since_last_record_factor : 33-68,mths_since_last_record_factor : 68-85,mths_since_last_record_factor : 75-85,mths_since_last_record_factor : >85
8321,867616,1081228,7800,7800,7800.00000,36 months,15.99,274.19,D,D2,...,0,0,0,0,0,0,0,0,0,1
21102,647751,828706,8000,8000,7500.00000,36 months,9.25,255.33,B,B2,...,0,0,0,0,0,0,0,0,0,1
23008,619833,794435,25000,17425,16710.15996,36 months,9.99,562.18,B,B4,...,0,0,0,0,0,0,0,0,0,1
15102,754302,954015,7100,7100,7100.00000,36 months,9.99,229.07,B,B1,...,0,0,0,0,0,0,0,0,0,1
12607,795194,999909,20375,20375,20350.00000,60 months,16.49,500.81,D,D3,...,0,0,0,0,0,0,0,0,0,1


In [4]:
loan_data_inputs_train.shape

(31773, 289)

In [5]:
loan_data_targets_train.shape

(31773, 1)

In [6]:
loan_data_inputs_test.shape

(7740, 289)

In [7]:
loan_data_targets_test.shape

(7740, 1)

### Selecting Features

In [8]:
inputs_train_with_ref_cat = loan_data_inputs_train.loc[: , ['grade : A',
'grade : B',
'grade : C',
'grade : D',
'grade : E',
'grade : F',
'grade : G',
'home_ownership : OWN_OTHER_NONE',
'home_ownership : MORTGAGE',
'home_ownership : RENT',
'addr_state : NE_SD_AK_NV_NM_FL_ND',
'addr_state : ID_TN_HI_MD_WA_NH',
'addr_state : CA',
'addr_state : UT_GA_NJ',
'addr_state : OR_KY_NC_SC_MI',
'addr_state : MO_OK_VT_MT_MN',
'addr_state : NY',
'addr_state : TX_AL_LA_KS_DC_DE_WY_IA_IN_ME',
'purpose : small_business',
'purpose : renewableEnergy_educational_house',
'purpose : other',
'purpose : debt_consolidation',
'purpose : medical_moving',
'purpose : homeimprovement_vacation_car',
'purpose : wedding_majorpurchase',
'purpose : credit_card',
'verification_status : Verified_SourceVerified',
'verification_status : Not Verified',
'initial_list_status : f',
'term_int : 36',
'term_int : 60',
'emp_length_int : 0',
'emp_length_int : 1',
'emp_length_int : 2-4',
'emp_length_int : 5-7',
'emp_length_int : 8-9',
'emp_length_int : >10',
'mnths_since_issue_d : <149',
'mnths_since_issue_d : 149-154',
'mnths_since_issue_d : 154-160',
'mnths_since_issue_d : 160-167',
'mnths_since_issue_d : 167-175',
'mnths_since_issue_d : 175-183',
'mnths_since_issue_d : >183',
'mnths_since_earliest_cr_line_factor : < 194',
'mnths_since_earliest_cr_line_factor : 194-213',
'mnths_since_earliest_cr_line_factor : 213-270',
'mnths_since_earliest_cr_line_factor : 270-366',
'mnths_since_earliest_cr_line_factor : 366-433',
'mnths_since_earliest_cr_line_factor : 433-490',
'mnths_since_earliest_cr_line_factor : > 490',
'delinq_2yrs_factor : 0',
'delinq_2yrs_factor : 1',
'delinq_2yrs_factor : 2-4',
'delinq_2yrs_factor : >=5',
'inq_last_6mths_factor : 0',
'inq_last_6mths_factor : 1-2',
'inq_last_6mths_factor : 3',
'inq_last_6mths_factor : >4',
'open_acc_factor : 0-4',
'open_acc_factor : 5-11',
'open_acc_factor : 12-15',
'open_acc_factor : 16-18',
'open_acc_factor : 16-20',
'open_acc_factor : >20',
'pub_rec_factor : 0',
'pub_rec_factor : 1',
'pub_rec_factor : >=2',
'total_acc_factor : 0-3',
'total_acc_factor : 4-8',
'total_acc_factor : 9-27',
'total_acc_factor : 28-39',
'total_acc_factor : 40-49',
'total_acc_factor : >=50',
'int_rate_factor : >6.179',
'int_rate_factor : 6.179-6.559',
'int_rate_factor : 6.559-8.077',
'int_rate_factor : 8.077-11.494',
'int_rate_factor : 11.494-16.049',
'int_rate_factor : 16.049-18.706',
'int_rate_factor : >18.706',
'annual_inc_factor : < 20K',
'annual_inc_factor : 20K-30K',
'annual_inc_factor : 30K-40K',
'annual_inc_factor : 40K-50K',
'annual_inc_factor : 50K-60K',
'annual_inc_factor : 60K-70K',
'annual_inc_factor : 70K-80K',
'annual_inc_factor : 80K-90K',
'annual_inc_factor : 90K-100K',
'annual_inc_factor : 100K-120K',
'annual_inc_factor : 120K-180K',
'annual_inc_factor : > 180K',
'dti_factor : < 1',
'dti_factor : 1-5',
'dti_factor : 5-7',
'dti_factor : 7-14',
'dti_factor : 14-16',
'dti_factor : 16-20',
'dti_factor : 20-24',
'dti_factor : 24-25',
'dti_factor : > 25',
'mths_since_last_delinq_factor : Missing',
'mths_since_last_delinq_factor : < 2',
'mths_since_last_delinq_factor : 3-19',
'mths_since_last_delinq_factor : 19-38',
'mths_since_last_delinq_factor : 38-52',
'mths_since_last_delinq_factor : 52-79',
'mths_since_last_delinq_factor : > 79',
'mths_since_last_record_factor : Missing',
'mths_since_last_record_factor : <4',
'mths_since_last_record_factor : 4-33',
'mths_since_last_record_factor : 33-37',
'mths_since_last_record_factor : 33-68',
'mths_since_last_record_factor : 68-85',
'mths_since_last_record_factor : 75-85',
'mths_since_last_record_factor : >85'
]]

In [9]:
ref_categories = [
'grade : G',
'home_ownership : OWN_OTHER_NONE',
'addr_state : NE_SD_AK_NV_NM_FL_ND',
'purpose : small_business',
'verification_status : Verified_SourceVerified',
'initial_list_status : f',
'term_int : 60',
'emp_length_int : 1',
'mnths_since_issue_d : <149',
'mnths_since_earliest_cr_line_factor : < 194',
'delinq_2yrs_factor : >=5',
'inq_last_6mths_factor : >4',
'open_acc_factor : 0-4',
'pub_rec_factor : >=2',
'total_acc_factor : 0-3',
'int_rate_factor : >18.706',
'annual_inc_factor : < 20K',
'dti_factor : > 25',
'mths_since_last_delinq_factor : < 2',
'mths_since_last_record_factor : <4'

]

In [10]:
inputs_train = inputs_train_with_ref_cat.drop(ref_categories,axis=1)
inputs_train.head()

,grade : A,grade : B,grade : C,grade : D,grade : E,grade : F,home_ownership : MORTGAGE,home_ownership : RENT,addr_state : ID_TN_HI_MD_WA_NH,addr_state : CA,...,mths_since_last_delinq_factor : 38-52,mths_since_last_delinq_factor : 52-79,mths_since_last_delinq_factor : > 79,mths_since_last_record_factor : Missing,mths_since_last_record_factor : 4-33,mths_since_last_record_factor : 33-37,mths_since_last_record_factor : 33-68,mths_since_last_record_factor : 68-85,mths_since_last_record_factor : 75-85,mths_since_last_record_factor : >85
8321,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
21102,0,1,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
23008,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
15102,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
12607,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
inputs_train.dtypes

grade : A                                int64
grade : B                                int64
grade : C                                int64
grade : D                                int64
grade : E                                int64
                                         ...  
mths_since_last_record_factor : 33-37    int64
mths_since_last_record_factor : 33-68    int64
mths_since_last_record_factor : 68-85    int64
mths_since_last_record_factor : 75-85    int64
mths_since_last_record_factor : >85      int64
Length: 97, dtype: object

# PD Model Estimation

## Logistic Regression

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [13]:
reg = LogisticRegression()

In [14]:
pd.options.display.max_rows = None

In [15]:
reg.fit(inputs_train,loan_data_targets_train)

LogisticRegression()

In [16]:
reg.intercept_

array([-0.52062304])

In [17]:
reg.coef_

array([[ 1.05009623,  0.67394391,  0.46761562,  0.22754506,  0.18394533,
        -0.01710547, -0.05932081,  0.00843433, -0.14112934, -0.2410854 ,
        -0.11533569,  0.06580859, -0.00302627,  0.04854928,  0.23328779,
         0.36762103,  0.55109803,  0.74504457,  0.54957255,  0.81786723,
         0.97568552,  0.96532082,  0.05834692,  0.        ,  0.        ,
         0.03594112, -0.03190995,  0.00996033,  0.1964955 ,  0.09570673,
         0.10964444,  0.12795239,  0.27731029,  0.38430332,  0.03674003,
        -0.0591209 , -0.03018922, -0.26458763, -0.26345646, -0.21920711,
        -0.28904089, -0.34893502, -0.11975504, -0.23602115,  0.55097938,
         0.35799196,  0.0989476 , -0.08833278, -0.11422448,  0.2517513 ,
        -0.22285265, -0.06469439,  0.        ,  0.        ,  0.06574399,
         0.0599654 , -0.03556176, -0.16564635, -0.19520079,  0.92673809,
        -0.18623769,  0.91039538,  0.6605055 ,  0.42490781,  0.20222825,
         0.09261813,  0.26536406,  0.47645618,  0.5

In [18]:
feature_name = inputs_train.columns.values

In [19]:
summary_table = pd.DataFrame(columns = ['Feature name'], data = feature_name)
summary_table['Coefficients'] = np.transpose(reg.coef_)
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept',reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature name,Coefficients
0,Intercept,-0.520623
1,grade : A,1.050096
2,grade : B,0.673944
3,grade : C,0.467616
4,grade : D,0.227545
5,grade : E,0.183945
6,grade : F,-0.017105
7,home_ownership : MORTGAGE,-0.059321
8,home_ownership : RENT,0.008434
9,addr_state : ID_TN_HI_MD_WA_NH,-0.141129


## Building a Logistic Model with P-values

#### OLS Stats Model for P-values

In [20]:
import numpy as np
import statsmodels.api as sm

# Add a constant to the inputs_train if you want to fit an intercept term
inputs_train_with_const = sm.add_constant(inputs_train)

# Fit the OLS model
ols_model = sm.OLS(loan_data_targets_train, inputs_train_with_const)
ols_results = ols_model.fit()

# Print summary statistics
ols_results.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               good_bad   R-squared:                       0.061
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     22.80
Date:                Tue, 28 May 2024   Prob (F-statistic):               0.00
Time:                        12:30:53   Log-Likelihood:                -10696.
No. Observations:               31773   AIC:                         2.158e+04
Df Residuals:                   31681   BIC:                         2.235e+04
Df Model:                          91                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                             0.4892      0.081      6.041      0.000       0.330       0.648
grade : A                                         0.1941      0.025      7.685      0.000       0.145       0.244
grade : B                                         0.1546      0.025      6.255      0.000       0.106       0.203
grade : C                                         0.1233      0.025      5.015      0.000       0.075       0.172
grade : D                                         0.0827      0.024      3.459      0.001       0.036       0.130
grade : E                                         0.0652      0.023      2.804      0.005       0.020       0.111
grade : F                                         0.0233      0.024      0.955      0.340      -0.025       0.071
home_ownership : MORTGAGE                        -0.0073      0.008     -0.963      0.336      -0.022       0.008
home_ownership : RENT                             0.0002      0.007      0.025      0.980      -0.014       0.015
addr_state : ID_TN_HI_MD_WA_NH                   -0.0157      0.009     -1.820      0.069      -0.033       0.001
addr_state : CA                                  -0.0290      0.006     -5.164      0.000      -0.040      -0.018
addr_state : UT_GA_NJ                            -0.0133      0.007     -1.844      0.065      -0.027       0.001
addr_state : OR_KY_NC_SC_MI                       0.0071      0.008      0.888      0.375      -0.009       0.023
addr_state : MO_OK_VT_MT_MN                       0.0009      0.010      0.090      0.928      -0.018       0.020
addr_state : NY                                   0.0057      0.007      0.808      0.419      -0.008       0.019
addr_state : TX_AL_LA_KS_DC_DE_WY_IA_IN_ME        0.0244      0.007      3.690      0.000       0.011       0.037
purpose : renewableEnergy_educational_house       0.0659      0.016      4.095      0.000       0.034       0.097
purpose : other                                   0.0858      0.011      7.935      0.000       0.065       0.107
purpose : debt_consolidation                      0.1092      0.009     11.599      0.000       0.091       0.128
purpose : medical_moving                          0.0855      0.014      6.151      0.000       0.058       0.113
purpose : homeimprovement_vacation_car            0.1159      0.011     11.037      0.000       0.095       0.137
purpose : wedding_majorpurchase                   0.1301      0.011     11.553      0.000       0.108       0.152
purpose : credit_card                             0.1316      0.010     12.553      0.000       0.111       0.152
verification_status : Not Verified                0.0069      0.004      1.604      0.109      -0.002       0.015
term_

#### Other

import statsmodels.api as sm

'' Assuming X_train and y_train are your training data
' Fit logistic regression model
logit_model = sm.Logit(loan_data_targets_train, sm.add_constant(inputs_train))

'' Fitting the model
#result = logit_model.fit()
result = logit_model.fit()

'' View the summary
result.summary()

from sklearn import linear_model
import scipy.stats as stat
from sklearn.utils.validation import column_or_1d

class LogisticRegression_with_p_values:

    def __init__(self, *args, **kwargs):
        self.model = linear_model.LogisticRegression(*args, **kwargs)

    def fit(self, X, y):
        y = column_or_1d(y, warn=True)
        self.model.fit(X, y)
        denom = (2.0 * (1.0 + np.cosh(self.model.decision_function(X))))
        denom = np.tile(denom, (X.shape[1], 1)).T
        F_ij = np.dot((X / denom).T, X)
        F_ij = F_ij.astype(np.float64)
        Cramer_Rao = np.linalg.inv(F_ij)
        try:
            Cramer_Rao = np.linalg.inv(F_ij)
        except np.linalg.LinAlgError:
            Cramer_Rao = np.zeros_like(F_ij)  
        sigma_estimates = np.sqrt(np.diagonal(Cramer_Rao))
        sigma_estimates[sigma_estimates == 0] = np.nan  
        sigma_estimates[np.isnan(sigma_estimates)] = 1.0  
        z_scores = self.model.coef_[0] / sigma_estimates
        p_values = [stat.norm.sf(abs(x)) * 2 for x in z_scores]
        self.coef_ = self.model.coef_
        self.intercept_ = self.model.intercept_
        self.p_values = p_values

reg = LogisticRegression_with_p_values()

reg.fit(inputs_train,loan_data_targets_train)

summary_table = pd.DataFrame(columns = ['Feature name'], data = feature_name)
summary_table['Coefficients'] = np.transpose(reg.coef_)
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept',reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

p_values = reg.p_values

p_values = np.append(np.nan,np.array(p_values))

summary_table['p_values'] = p_values

summary_table

inputs_train_with_ref_cat = loan_data_inputs_train.loc[ : , ['loan_status : Charged Off',
'loan_status : Current',
'loan_status : Fully Paid'
]]

ref_categories = [
    'loan_status : Charged Off'
]

inputs_train = inputs_train_with_ref_cat.drop(ref_categories,axis=1)
inputs_train.head()

reg2 = LogisticRegression_with_p_values()
reg2.fit(inputs_train,loan_data_targets_train)

feature_name = inputs_train.columns.values

summary_table = pd.DataFrame(columns = ['Feature name'], data = feature_name)
summary_table['Coefficients'] = np.transpose(reg2.coef_)
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept',reg2.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

p_values = reg2.p_values
p_values = np.append(np.nan,np.array(p_values))
summary_table['p_values'] = p_values
summary_table

# PD Model Validation

### Out-of-sample validation (test)

In [21]:
inputs_train_with_ref_cat = loan_data_inputs_train.loc[: , ['grade : A',
'grade : B',
'grade : C',
'grade : D',
'grade : E',
'grade : F',
'grade : G',
'home_ownership : OWN_OTHER_NONE',
'home_ownership : MORTGAGE',
'home_ownership : RENT',
'addr_state : NE_SD_AK_NV_NM_FL_ND',
'addr_state : ID_TN_HI_MD_WA_NH',
'addr_state : CA',
'addr_state : UT_GA_NJ',
'addr_state : OR_KY_NC_SC_MI',
'addr_state : MO_OK_VT_MT_MN',
'addr_state : NY',
'addr_state : TX_AL_LA_KS_DC_DE_WY_IA_IN_ME',
'purpose : small_business',
'purpose : renewableEnergy_educational_house',
'purpose : other',
'purpose : debt_consolidation',
'purpose : medical_moving',
'purpose : homeimprovement_vacation_car',
'purpose : wedding_majorpurchase',
'purpose : credit_card',
'verification_status : Verified_SourceVerified',
'verification_status : Not Verified',
'initial_list_status : f',
'term_int : 36',
'term_int : 60',
'emp_length_int : 0',
'emp_length_int : 1',
'emp_length_int : 2-4',
'emp_length_int : 5-7',
'emp_length_int : 8-9',
'emp_length_int : >10',
'mnths_since_issue_d : <149',
'mnths_since_issue_d : 149-154',
'mnths_since_issue_d : 154-160',
'mnths_since_issue_d : 160-167',
'mnths_since_issue_d : 167-175',
'mnths_since_issue_d : 175-183',
'mnths_since_issue_d : >183',
'mnths_since_earliest_cr_line_factor : < 194',
'mnths_since_earliest_cr_line_factor : 194-213',
'mnths_since_earliest_cr_line_factor : 213-270',
'mnths_since_earliest_cr_line_factor : 270-366',
'mnths_since_earliest_cr_line_factor : 366-433',
'mnths_since_earliest_cr_line_factor : 433-490',
'mnths_since_earliest_cr_line_factor : > 490',
'delinq_2yrs_factor : 0',
'delinq_2yrs_factor : 1',
'delinq_2yrs_factor : 2-4',
'delinq_2yrs_factor : >=5',
'inq_last_6mths_factor : 0',
'inq_last_6mths_factor : 1-2',
'inq_last_6mths_factor : 3',
'inq_last_6mths_factor : >4',
'open_acc_factor : 0-4',
'open_acc_factor : 5-11',
'open_acc_factor : 12-15',
'open_acc_factor : 16-18',
'open_acc_factor : 16-20',
'open_acc_factor : >20',
'pub_rec_factor : 0',
'pub_rec_factor : 1',
'pub_rec_factor : >=2',
'total_acc_factor : 0-3',
'total_acc_factor : 4-8',
'total_acc_factor : 9-27',
'total_acc_factor : 28-39',
'total_acc_factor : 40-49',
'total_acc_factor : >=50',
'int_rate_factor : >6.179',
'int_rate_factor : 6.179-6.559',
'int_rate_factor : 6.559-8.077',
'int_rate_factor : 8.077-11.494',
'int_rate_factor : 11.494-16.049',
'int_rate_factor : 16.049-18.706',
'int_rate_factor : >18.706',
'annual_inc_factor : < 20K',
'annual_inc_factor : 20K-30K',
'annual_inc_factor : 30K-40K',
'annual_inc_factor : 40K-50K',
'annual_inc_factor : 50K-60K',
'annual_inc_factor : 60K-70K',
'annual_inc_factor : 70K-80K',
'annual_inc_factor : 80K-90K',
'annual_inc_factor : 90K-100K',
'annual_inc_factor : 100K-120K',
'annual_inc_factor : 120K-180K',
'annual_inc_factor : > 180K',
'dti_factor : < 1',
'dti_factor : 1-5',
'dti_factor : 5-7',
'dti_factor : 7-14',
'dti_factor : 14-16',
'dti_factor : 16-20',
'dti_factor : 20-24',
'dti_factor : 24-25',
'dti_factor : > 25',
'mths_since_last_delinq_factor : Missing',
'mths_since_last_delinq_factor : < 2',
'mths_since_last_delinq_factor : 3-19',
'mths_since_last_delinq_factor : 19-38',
'mths_since_last_delinq_factor : 38-52',
'mths_since_last_delinq_factor : 52-79',
'mths_since_last_delinq_factor : > 79',
'mths_since_last_record_factor : Missing',
'mths_since_last_record_factor : <4',
'mths_since_last_record_factor : 4-33',
'mths_since_last_record_factor : 33-37',
'mths_since_last_record_factor : 33-68',
'mths_since_last_record_factor : 68-85',
'mths_since_last_record_factor : 75-85',
'mths_since_last_record_factor : >85'
]]

In [22]:
ref_categories = [
'grade : G',
'home_ownership : OWN_OTHER_NONE',
'addr_state : NE_SD_AK_NV_NM_FL_ND',
'purpose : small_business',
'verification_status : Verified_SourceVerified',
'initial_list_status : f',
'term_int : 60',
'emp_length_int : 1',
'mnths_since_issue_d : <149',
'mnths_since_earliest_cr_line_factor : < 194',
'delinq_2yrs_factor : >=5',
'inq_last_6mths_factor : >4',
'open_acc_factor : 0-4',
'pub_rec_factor : >=2',
'total_acc_factor : 0-3',
'int_rate_factor : >18.706',
'annual_inc_factor : < 20K',
'dti_factor : > 25',
'mths_since_last_delinq_factor : < 2',
'mths_since_last_record_factor : <4'
]]

In [23]:
inputs_test = inputs_test_with_ref_cat.drop(ref_categories,axis=1)
inputs_test.head()

,grade : A,grade : B,grade : C,grade : D,grade : E,grade : F,purpose : renewableEnergy_educational_house,purpose : other,purpose : debt_consolidation,purpose : medical_moving,...,annual_inc_factor : 30K-40K,annual_inc_factor : 40K-50K,annual_inc_factor : 50K-60K,annual_inc_factor : 60K-70K,annual_inc_factor : 70K-80K,annual_inc_factor : 80K-90K,annual_inc_factor : 90K-100K,annual_inc_factor : 100K-120K,annual_inc_factor : 120K-180K,annual_inc_factor : > 180K
36705,False,False,True,False,False,False,False,False,False,False,...,0,1,0,0,0,0,0,0,0,0
21671,False,True,False,False,False,False,False,False,False,False,...,0,0,1,0,0,0,0,0,0,0
30630,False,False,False,False,True,False,False,False,True,False,...,0,0,0,0,1,0,0,0,0,0
36198,False,False,False,True,False,False,False,False,True,False,...,0,0,0,0,1,0,0,0,0,0
4050,True,False,False,False,False,False,False,False,True,False,...,0,0,0,0,1,0,0,0,0,0


In [24]:
train_columns = inputs_train_with_ref_cat.columns

# Check for missing columns in test data
missing_cols = set(train_columns) - set(inputs_test_with_ref_cat.columns)
if missing_cols:
    print(f"Missing columns in test data: {missing_cols}")

# Check for additional columns in test data
extra_cols = set(inputs_test_with_ref_cat.columns) - set(train_columns)
if extra_cols:
    print(f"Extra columns in test data: {extra_cols}")

Missing columns in test data: {'delinq_2yrs_factor : 0', 'dti_factor : 1-5', 'mths_since_last_delinq_factor : 52-79', 'mths_since_last_record_factor : Missing', 'dti_factor : 20-24', 'delinq_2yrs_factor : 1', 'addr_state : NE_SD_AK_NV_NM_FL_ND', 'mnths_since_earliest_cr_line_factor : < 194', 'mnths_since_earliest_cr_line_factor : 194-213', 'mths_since_last_record_factor : <4', 'dti_factor : 5-7', 'mnths_since_earliest_cr_line_factor : 433-490', 'emp_length_int : 2-4', 'emp_length_int : >10', 'dti_factor : 16-20', 'emp_length_int : 8-9', 'emp_length_int : 0', 'dti_factor : 24-25', 'pub_rec_factor : 1', 'emp_length_int : 5-7', 'total_acc_factor : 4-8', 'mths_since_last_delinq_factor : Missing', 'addr_state : TX_AL_LA_KS_DC_DE_WY_IA_IN_ME', 'open_acc_factor : 5-11', 'open_acc_factor : >20', 'open_acc_factor : 0-4', 'mths_since_last_delinq_factor : > 79', 'home_ownership : OWN_OTHER_NONE', 'mths_since_last_record_factor : 33-68', 'open_acc_factor : 16-20', 'total_acc_factor : 40-49', 'addr

In [25]:
# Align the test data columns to match the training data columns
inputs_test_aligned = inputs_test_with_ref_cat.reindex(columns=train_columns, fill_value=0)

# Verify the alignment
inputs_test_aligned.head()

,grade : A,grade : B,grade : C,grade : D,grade : E,grade : F,grade : G,home_ownership : OWN_OTHER_NONE,home_ownership : MORTGAGE,home_ownership : RENT,...,mths_since_last_delinq_factor : 52-79,mths_since_last_delinq_factor : > 79,mths_since_last_record_factor : Missing,mths_since_last_record_factor : <4,mths_since_last_record_factor : 4-33,mths_since_last_record_factor : 33-37,mths_since_last_record_factor : 33-68,mths_since_last_record_factor : 68-85,mths_since_last_record_factor : 75-85,mths_since_last_record_factor : >85
36705,False,False,True,False,False,False,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21671,False,True,False,False,False,False,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30630,False,False,False,False,True,False,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36198,False,False,False,True,False,False,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4050,True,False,False,False,False,False,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# Define reference categories

# Drop reference categories from both training and test data
inputs_train_aligned = inputs_train_with_ref_cat.drop(ref_categories, axis=1)
inputs_test_aligned = inputs_test_aligned.drop(ref_categories, axis=1)

# Verify the dataframes
inputs_train_aligned.head()

,grade : A,grade : B,grade : C,grade : D,grade : E,grade : F,home_ownership : OWN_OTHER_NONE,home_ownership : MORTGAGE,home_ownership : RENT,addr_state : NE_SD_AK_NV_NM_FL_ND,...,mths_since_last_delinq_factor : 52-79,mths_since_last_delinq_factor : > 79,mths_since_last_record_factor : Missing,mths_since_last_record_factor : <4,mths_since_last_record_factor : 4-33,mths_since_last_record_factor : 33-37,mths_since_last_record_factor : 33-68,mths_since_last_record_factor : 68-85,mths_since_last_record_factor : 75-85,mths_since_last_record_factor : >85
8321,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
21102,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
23008,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
15102,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
12607,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [27]:
inputs_test_aligned.head()

,grade : A,grade : B,grade : C,grade : D,grade : E,grade : F,home_ownership : OWN_OTHER_NONE,home_ownership : MORTGAGE,home_ownership : RENT,addr_state : NE_SD_AK_NV_NM_FL_ND,...,mths_since_last_delinq_factor : 52-79,mths_since_last_delinq_factor : > 79,mths_since_last_record_factor : Missing,mths_since_last_record_factor : <4,mths_since_last_record_factor : 4-33,mths_since_last_record_factor : 33-37,mths_since_last_record_factor : 33-68,mths_since_last_record_factor : 68-85,mths_since_last_record_factor : 75-85,mths_since_last_record_factor : >85
36705,False,False,True,False,False,False,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21671,False,True,False,False,False,False,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30630,False,False,False,False,True,False,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36198,False,False,False,True,False,False,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4050,True,False,False,False,False,False,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:

# Predict using the aligned test data
y_hat_test = reg.predict(inputs_test_aligned)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- addr_state : NE_SD_AK_NV_NM_FL_ND
- delinq_2yrs_factor : >=5
- dti_factor : > 25
- emp_length_int : 1
- home_ownership : OWN_OTHER_NONE
- ...


In [ ]:
y_hat_test

In [ ]:
y_hat_test_proba = reg2.model.predict_proba(inputs_test_aligned)

In [ ]:
y_hat_test_proba

In [ ]:
y_hat_test_proba[:][: , 1]

In [ ]:
y_hat_test_proba = y_hat_test_proba[:][: , 1]

In [ ]:
y_hat_test_proba

In [ ]:
loan_data_targets_test_temp = loan_data_targets_test

In [ ]:
loan_data_targets_test_temp.reset_index(drop=True, inplace =True)

In [ ]:
df_actual_predicted_probs = pd.concat([loan_data_targets_test_temp,pd.DataFrame(y_hat_test_proba)],axis=1)

In [ ]:
df_actual_predicted_probs.shape

In [ ]:
df_actual_predicted_probs.columns = ['loan_data_targets_test','y_hat_test_proba']

In [ ]:
df_actual_predicted_probs.index = loan_data_inputs_test.index

In [ ]:
df_actual_predicted_probs.head()

### Accuracy and Area under the curve

In [ ]:
# Default Threshold
tr = 0.5 
# 0.5 gives false positives but high accuracy, and 0.9(more conservative) gives less accuracy but lesser flase positives
df_actual_predicted_probs['y_hat_test'] = np.where(df_actual_predicted_probs['y_hat_test_proba'] > tr ,1 ,0)

In [ ]:
pd.crosstab(df_actual_predicted_probs['loan_data_targets_test'],df_actual_predicted_probs['y_hat_test'],rownames = ['Actual'],colnames = ['Predicted'])

In [ ]:
pd.crosstab(df_actual_predicted_probs['loan_data_targets_test'],df_actual_predicted_probs['y_hat_test'],rownames = ['Actual'],colnames = ['Predicted']) / df_actual_predicted_probs.shape[0] 

In [ ]:
(pd.crosstab(df_actual_predicted_probs['loan_data_targets_test'],df_actual_predicted_probs['y_hat_test'],rownames = ['Actual'],colnames = ['Predicted']) / df_actual_predicted_probs.shape[0]).iloc[0,0] + (pd.crosstab(df_actual_predicted_probs['loan_data_targets_test'],df_actual_predicted_probs['y_hat_test'],rownames = ['Actual'],colnames = ['Predicted']) / df_actual_predicted_probs.shape[0]).iloc[1,1]  

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
roc_curve(df_actual_predicted_probs['loan_data_targets_test'],df_actual_predicted_probs['y_hat_test_proba'])

In [ ]:
fpr,tpr,thresholds = roc_curve(df_actual_predicted_probs['loan_data_targets_test'],df_actual_predicted_probs['y_hat_test_proba'])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
plt.plot(fpr,tpr)
plt.plot(fpr,fpr,linestyle = '--', color = 'k')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')

In [ ]:
AUROC = roc_auc_score(df_actual_predicted_probs['loan_data_targets_test'],df_actual_predicted_probs['y_hat_test_proba'])
AUROC

In [ ]:
df_actual_predicted_probs = df_actual_predicted_probs.reset_index()

In [ ]:
df_actual_predicted_probs.head()

In [ ]:
df_actual_predicted_probs['Cumulative N Population'] = df_actual_predicted_probs.index + 1
df_actual_predicted_probs['Cumulative N Good'] = df_actual_predicted_probs['loan_data_targets_test'].cumsum()
df_actual_predicted_probs['Cumulative N Bad'] = df_actual_predicted_probs['Cumulative N Population'] - df_actual_predicted_probs['loan_data_targets_test'].cumsum()

In [ ]:
df_actual_predicted_probs['Cumulative Perc Population'] = df_actual_predicted_probs['Cumulative N Population'] / (df_actual_predicted_probs.shape[0])
df_actual_predicted_probs['Cumulative Perc Good'] = df_actual_predicted_probs['Cumulative N Good'] / df_actual_predicted_probs['loan_data_targets_test'].sum()
df_actual_predicted_probs['Cumulative Perc Bad'] = df_actual_predicted_probs['Cumulative N Bad'] /  (df_actual_predicted_probs.shape[0] - df_actual_predicted_probs['loan_data_targets_test'].sum())

In [ ]:
df_actual_predicted_probs.head()

In [ ]:
df_actual_predicted_probs.tail()

In [ ]:
plt.plot(df_actual_predicted_probs['Cumulative Perc Population'], df_actual_predicted_probs['Cumulative Perc Bad'])
plt.plot(df_actual_predicted_probs['Cumulative Perc Population'],df_actual_predicted_probs['Cumulative Perc Population'],linestyle = '--', color = 'k')
plt.xlabel('Cumulative % Population')
plt.ylabel('Cumulative % Bad')
plt.title('Gini')

In [ ]:
Gini = AUROC*2 -1
Gini

In [ ]:
plt.plot(df_actual_predicted_probs['y_hat_test_proba'], df_actual_predicted_probs['Cumulative Perc Bad'] ,color = 'r')
plt.plot(df_actual_predicted_probs['y_hat_test_proba'], df_actual_predicted_probs['Cumulative Perc Good'] ,color = 'b')
plt.xlabel('Estimated Probability for being Good')
plt.ylabel('Cumulative %')
plt.title('Kolmogorov-Smirnov')

In [ ]:
KS = max(df_actual_predicted_probs['Cumulative Perc Bad'] - df_actual_predicted_probs['Cumulative Perc Good'])
KS

# Applying the PD Model

### Calculating the PD of individual accounts

In [ ]:
pd.options.display.max_columns = None

In [ ]:
inputs_test_with_ref_cat.head()

In [ ]:
summary_table

In [ ]:
y_hat_test_proba

### Creating a Scorecard

In [ ]:
summary_table

In [ ]:
ref_categories

In [ ]:
df_ref_categories = pd.DataFrame(ref_categories, columns = ['Feature Name'])
df_ref_categories['Coefficients'] = 0 
df_ref_categories['p_values'] = np.nan
df_ref_categories

In [ ]:
df_scorecard = pd.concat([summary_table,df_ref_categories])
df_scorecard = df_scorecard.reset_index()
df_scorecard

In [ ]:
df_scorecard['Original feature name'] = df_scorecard['Feature Name'].str.split(' : ').str[0]

In [ ]:
min_score = 300
max_score = 850

In [ ]:
df_scorecard.groupby('Original feature name')['Coefficients'].min()

In [ ]:
min_sum_coeff = df_scorecard.groupby('Original feature name')['Coefficients'].min().sum()
min_sum_coeff

In [ ]:
df_scorecard.groupby('Original feature name')['Coefficients'].max()

In [ ]:
max_sum_coeff = df_scorecard.groupby('Original feature name')['Coefficients'].max().sum()
max_sum_coeff

In [ ]:
df_scorecard['Score - Calculation'] = df_scorecard['Coefficients'] * (max_score - min_score) / (max_sum_coeff - min_sum_coeff)
df_scorecard

In [ ]:
df_scorecard['Score - Calculation'][0] = ((df_scorecard['Coefficients'][0] - min_sum_coeff) / (max_sum_coeff - min_sum_coeff)) * (max_score-min_score) + min_score
df_scorecard

In [ ]:
df_scorecard['Score  - Prelimiary'] = df_scorecard['Score - Calculation'].round()
df_scorecard

In [ ]:
min_sum_score_prel = df_scorecard.groupby('Original feature name')['Score  - Prelimiary'].min().sum()
min_sum_score_prel

In [ ]:
max_sum_score_prel = df_scorecard.groupby('Original feature name')['Score  - Prelimiary'].max().sum()
max_sum_score_prel

In [ ]:
df_scorecard['Difference'] = df_scorecard['Score  - Prelimiary'] - df_scorecard['Score - Calculation']
df_scorecard

In [ ]:
df_scorecard['Score - Final'] = df_scorecard['Score  - Prelimiary']
df_scorecard['Score - Final'][77] = 16
df_scorecard

In [ ]:
min_sum_score_prel = df_scorecard.groupby('Original feature name')['Score - Final'].min().sum()
min_sum_score_prel

In [ ]:
max_sum_score_prel = df_scorecard.groupby('Original feature name')['Score - Final'].max().sum()
max_sum_score_prel

### Calculating a Credit Score

In [ ]:
inputs_test_with_ref_cat.head()

In [ ]:
df_scorecard

In [ ]:
inputs_test_with_ref_cat_w_intercept = inputs_test_with_ref_cat

In [ ]:
inputs_test_with_ref_cat_w_intercept.insert(0, 'Intercept',1)

In [ ]:
inputs_test_with_ref_cat_w_intercept.head()

In [ ]:
inputs_test_with_ref_cat_w_intercept = inputs_test_with_ref_cat_w_intercept[df_scorecard['Feature name'].values]

In [ ]:
inputs_test_with_ref_cat_w_intercept.head()

In [ ]:
scorecard_scores = df_scorecard['Score - Final']

In [ ]:
inputs_test_with_ref_cat_w_intercept.shape

In [ ]:
scorecard_scores.shape

In [ ]:
scorecard_scores = scorecard_scores.values.reshape()

In [ ]:
scorecard_scores.shape

In [ ]:
y_scores = inputs_test_with_ref_cat_w_intercept.dot(scorecard_scores)

In [ ]:
y_scores.head()

### From Credit Score to PD

In [ ]:
sum_coef_from_score = ((y_scores - min_score) / (max_score - min_score)) * (max_sum_coef) + min_sum_coef

In [ ]:
y_hat_proba_from_score = np.exp(sum_coeff_from_score)
y_hat_proba_from_score.head()

In [ ]:
y_hat_proba_test_proba[0: 5]
#Difference because we rounded them

### Setting Cutoffs

In [ ]:
# Default Threshold
tr = 0.5 
# 0.5 gives false positives but high accuracy, and 0.9(more conservative) gives less accuracy but lesser flase positives
df_actual_predicted_probs['y_hat_test'] = np.where(df_actual_predicted_probs['y_hat_test_proba'] > tr ,1 ,0)

In [ ]:
pd.crosstab(df_actual_predicted_probs['loan_data_targets_test'],df_actual_predicted_probs['y_hat_test'],rownames = ['Actual'],colnames = ['Predicted'])

In [ ]:
pd.crosstab(df_actual_predicted_probs['loan_data_targets_test'],df_actual_predicted_probs['y_hat_test'],rownames = ['Actual'],colnames = ['Predicted']) / df_actual_predicted_probs.shape[0] 

In [ ]:
(pd.crosstab(df_actual_predicted_probs['loan_data_targets_test'],df_actual_predicted_probs['y_hat_test'],rownames = ['Actual'],colnames = ['Predicted']) / df_actual_predicted_probs.shape[0]).iloc[0,0] + (pd.crosstab(df_actual_predicted_probs['loan_data_targets_test'],df_actual_predicted_probs['y_hat_test'],rownames = ['Actual'],colnames = ['Predicted']) / df_actual_predicted_probs.shape[0]).iloc[1,1]  

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
roc_curve(df_actual_predicted_probs['loan_data_targets_test'],df_actual_predicted_probs['y_hat_test_proba'])

In [ ]:
fpr,tpr,thresholds = roc_curve(df_actual_predicted_probs['loan_data_targets_test'],df_actual_predicted_probs['y_hat_test_proba'])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
plt.plot(fpr,tpr)
plt.plot(fpr,fpr,linestyle = '--', color = 'k')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')

In [ ]:
thresholds.shape

In [ ]:
df_cutoffs = pd.concat([pd.DataFrame(thresholds),pd.DataFrame(tpr)],axis=1)

In [ ]:
df_cutoffs.columns = ['thresholds', 'fpr', 'tpr']

In [ ]:
df_cutoffs.head()

In [ ]:
df_cutoffs['threshold'][0] = 1 - 1 / np.power(10,16)

In [ ]:
df_cutoffs['Score'] = ((np.log(df_cutoffs['thresholds'] / (1 - df_cutoffs['thresholds'])) - min_sum_coef) *((max_score - min_score) / (max_sum_coef - min_sum_coef)) + min_score).round()

In [ ]:
df_cutoffs['Score'][0] = max_score

In [ ]:
df_cutoffs.head()

In [ ]:
df_cutoffs.tail()

In [ ]:
def n_approved(p):
    return np.where(df_actual_predicted_probs['y_hat_test_proba'] >= p , 1, 0).sum()

In [ ]:
df_cutoffs['N Aprroved'] = df_cutoffs['thresholds'].apply(n_approved)
df_cutoffs['N Rejected'] = df_actual_predicted_probs['y_hat_test_proba'].shape[0] - df_cutoffs['N Aprroved']
df_cutoffs['Aprroval Rate'] = df_cutoffs['N Aprroved'] / df_actual_predicted_probs['y_hat_test_proba'].shape[0]
df_cutoffs['Rejection Rate'] = 1 - df_cutoffs['Aprroval Rate']

In [ ]:
df_cutoffs.head()

In [ ]:
df_cutoffs.tail()

PD with 10%

In [ ]:
df_cutoffs.iloc[5000: 6200,]

PD with 5%

In [ ]:
df_cutoffs.iloc[1000: 2000,]

In [ ]:
inputs_train_with_ref_cat.to_csv('inputs_train_with_ref_cat.csv')

In [ ]:
df_scorecard.to_csv('df_scorecard.csv')